In [1]:
!pip install faiss-cpu numpy sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 2.9 MB/s eta 0:00:00a 0:00:01


In [2]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

print(f"Versión de FAISS: {faiss.__version__}")

/opt/anaconda3/envs/agents/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Versión de FAISS: 1.11.0


In [3]:
print("Cargando modelo de embedding...")
embed_model = SentenceTransformer("all-MiniLM-L6-v2") # Este modelo genera embeddings de 384 dimensiones
dimension = 384 # Dimensión de los embeddings generados por all-MiniLM-L6-v2
print(f"Modelo cargado. Dimensión de los embeddings: {dimension}")

Cargando modelo de embedding...
Modelo cargado. Dimensión de los embeddings: 384


In [4]:
sentences = [
    "El gato se sentó en la alfombra.",
    "El perro jugaba en el jardín.",
    "La inteligencia artificial está transformando el mundo.",
    "FAISS es una biblioteca para búsqueda de similitud eficiente.",
    "Los modelos de lenguaje grandes son muy potentes.",
    "Amo programar en Python.",
    "El sol brilla intensamente hoy.",
    "La búsqueda de vectores es importante para RAG."
]

In [5]:
print("Generando embeddings para las frases de ejemplo...")
embeddings = embed_model.encode(sentences)
# Asegurarse de que los embeddings sean de tipo float32, que es lo que FAISS espera comúnmente
embeddings = np.array(embeddings).astype("float32")
num_vectors = embeddings.shape[0]
print(f"Se generaron {num_vectors} embeddings, cada uno de dimensión {embeddings.shape[1]}.")
# print(embeddings)

Generando embeddings para las frases de ejemplo...
Se generaron 8 embeddings, cada uno de dimensión 384.


In [6]:
print(f"\nCreando índice FAISS (IndexFlatL2) para dimensión {dimension}...")
index_flat_l2 = faiss.IndexFlatL2(dimension)


Creando índice FAISS (IndexFlatL2) para dimensión 384...


In [7]:
print(f"¿Índice entrenado? {index_flat_l2.is_trained}") # Debería ser True

¿Índice entrenado? True


In [8]:
print(f"Añadiendo {num_vectors} vectores al índice...")
index_flat_l2.add(embeddings)

Añadiendo 8 vectores al índice...


In [9]:
print(f"Número total de vectores en el índice: {index_flat_l2.ntotal}")

Número total de vectores en el índice: 8


In [10]:
query_sentence = "IA y modelos de lenguaje"
print(f"\nGenerando embedding para la frase de consulta: '{query_sentence}'")
query_embedding = embed_model.encode([query_sentence])
query_embedding = np.array(query_embedding).astype("float32")

# Número de vecinos más cercanos a encontrar
k = 3 
print(f"Buscando los {k} vecinos más cercanos...")

# El método search devuelve dos arrays: D (distancias) e I (índices)
# D: array de forma (num_queries, k) con las distancias L2 al cuadrado
# I: array de forma (num_queries, k) con los índices de los vecinos más cercanos en la base de datos original

Distances_flat, Indices_flat = index_flat_l2.search(query_embedding, k)

print("\nResultados de la búsqueda con IndexFlatL2:")
for i in range(k):
    idx = Indices_flat[0][i]
    dist = Distances_flat[0][i]
    print(f"  Vecino {i+1}: Índice={idx}, Distancia L2^2={dist:.4f}, Frase Original: '{sentences[idx]}'")


Generando embedding para la frase de consulta: 'IA y modelos de lenguaje'
Buscando los 3 vecinos más cercanos...

Resultados de la búsqueda con IndexFlatL2:
  Vecino 1: Índice=4, Distancia L2^2=0.3986, Frase Original: 'Los modelos de lenguaje grandes son muy potentes.'
  Vecino 2: Índice=2, Distancia L2^2=0.9251, Frase Original: 'La inteligencia artificial está transformando el mundo.'
  Vecino 3: Índice=6, Distancia L2^2=1.1198, Frase Original: 'El sol brilla intensamente hoy.'


In [1]:
print("\n--- Explorando IndexIVFFlat ---")
# Número de celdas (listas invertidas). Un buen valor es sqrt(num_vectors) a 4*sqrt(num_vectors)
nlist = int(np.sqrt(num_vectors)) # Ejemplo: para 8 vectores, sqrt(8) ~ 2 o 3. Usaremos 2 para este pequeño ejemplo.
if nlist == 0: nlist = 1 # Asegurar al menos 1

print(f"Creando cuantizador (IndexFlatL2) para IndexIVFFlat...")
quantizer = faiss.IndexFlatL2(dimension) # El cuantizador es otro índice, usado para asignar vectores a celdas

print(f"Creando índice IndexIVFFlat con {nlist} celdas...")
index_ivf_flat = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_L2)
# faiss.METRIC_L2 especifica la métrica de distancia (también existe faiss.METRIC_INNER_PRODUCT)

# Entrenar el índice IVFFlat
# Necesita ver una muestra de los datos (o todos los datos si son pocos) para aprender los centroides.
print(f"Entrenando el índice IndexIVFFlat con {num_vectors} vectores...")
if not index_ivf_flat.is_trained:
    index_ivf_flat.train(embeddings)
print(f"¿Índice IVFFlat entrenado? {index_ivf_flat.is_trained}")

# Añadir los vectores al índice IVFFlat
print(f"Añadiendo {num_vectors} vectores al índice IVFFlat...")
index_ivf_flat.add(embeddings)
print(f"Número total de vectores en el índice IVFFlat: {index_ivf_flat.ntotal}")

# Realizar una búsqueda con IndexIVFFlat
# nprobe: cuántas celdas cercanas visitar durante la búsqueda. Aumentar mejora la precisión pero reduce la velocidad.
index_ivf_flat.nprobe = 1 # Para este ejemplo pequeño, 1 puede ser suficiente. Para datos más grandes, se ajusta.
print(f"Buscando los {k} vecinos más cercanos con IndexIVFFlat (nprobe={index_ivf_flat.nprobe})...")

Distances_ivf, Indices_ivf = index_ivf_flat.search(query_embedding, k)

print("\nResultados de la búsqueda con IndexIVFFlat:")
for i in range(k):
    idx = Indices_ivf[0][i]
    dist = Distances_ivf[0][i]
    # A veces, si k es mayor que los elementos en las celdas visitadas, puede devolver -1 como índice
    if idx != -1:
        print(f"  Vecino {i+1}: Índice={idx}, Distancia L2^2={dist:.4f}, Frase Original: '{sentences[idx]}'")
    else:
        print(f"  Vecino {i+1}: No se encontró (índice -1)")

print("\nNota: Con IndexIVFFlat y pocos datos/celdas, los resultados pueden variar o ser menos precisos que IndexFlatL2.")
print("Su ventaja se ve en conjuntos de datos mucho más grandes.")


--- Explorando IndexIVFFlat ---


NameError: name 'np' is not defined